# Objective
This is the first notebook in my **Chicago Traffic Crashes** data analysis project, and in the notebook I will be **importing** and **merging** the provided datasets.

### Order of Operations:
* Obtain data
* Explore the datasets we will be using to create our Models i.e, **Trafic_Crashes, Traffic_People,** and **Traffic_Vehicles**
    - Filter out irrelevant columns
    - Data type conversion
    - Detect and handle missing data
    - Deal with outliers in data if any
    - Check for duplicates
    - Create dummy variables for categorical data

## Data Importing
The first step in this process is to load obtained data into the notebook.

In [1]:
# First we need to import all necessary libraries

# For interacting with path/directory
import os 
from glob import glob

import pandas as pd  # For handle datasets and data storage
import sqlite3  # For joining datasets

# For data visualization
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

# Import functions.py
from mydef import *

### Traffic_Crashes - Load and Explore Dataset

In [2]:
# Start by importing and exploring the first dataset
crashes = pd.read_csv('CSV_Datasets/Traffic_Crashes.csv')
crashes.head().T

,0,1,2,3,4
CRASH_RECORD_ID,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...
RD_NO,JC343143,JA329216,JD292400,JD293602,JD290451
CRASH_DATE_EST_I,NaN,NaN,NaN,NaN,NaN
CRASH_DATE,07/10/2019 05:56:00 PM,06/30/2017 04:00:00 PM,07/10/2020 10:25:00 AM,07/11/2020 01:00:00 AM,07/08/2020 02:00:00 PM
POSTED_SPEED_LIMIT,35,35,30,30,20
TRAFFIC_CONTROL_DEVICE,NO CONTROLS,STOP SIGN/FLASHER,TRAFFIC SIGNAL,NO CONTROLS,NO CONTROLS
DEVICE_CONDITION,NO CONTROLS,FUNCTIONING PROPERLY,FUNCTIONING PROPERLY,NO CONTROLS,NO CONTROLS
WEATHER_CONDITION,CLEAR,CLEAR,CLEAR,CLEAR,CLEAR
LIGHTING_CONDITION,DAYLIGHT,DAYLIGHT,DAYLIGHT,DARKNESS,DAYLIGHT
FIRST_CRASH_TYPE,TURNING,TURNING,REAR END,PARKED MOTOR VEHICLE,PARKED MOTOR VEHICLE


In [3]:
# View size of dataset
print(f'This dataset has {len(crashes)} rows and {len(crashes.columns)} columns\n')

# .info() to get more information on each column, i.e, dtypes, number of non-null values
crashes.info()

This dataset has 490128 rows and 49 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490128 entries, 0 to 490127
Data columns (total 49 columns):
CRASH_RECORD_ID                  490128 non-null object
RD_NO                            486363 non-null object
CRASH_DATE_EST_I                 36817 non-null object
CRASH_DATE                       490128 non-null object
POSTED_SPEED_LIMIT               490128 non-null int64
TRAFFIC_CONTROL_DEVICE           490128 non-null object
DEVICE_CONDITION                 490128 non-null object
WEATHER_CONDITION                490128 non-null object
LIGHTING_CONDITION               490128 non-null object
FIRST_CRASH_TYPE                 490128 non-null object
TRAFFICWAY_TYPE                  490128 non-null object
LANE_CNT                         198965 non-null float64
ALIGNMENT                        490128 non-null object
ROADWAY_SURFACE_COND             490128 non-null object
ROAD_DEFECT                      490128 non-null object
REPOR

A couple of things we know so far:
* There are 482,332 rows and 49 columns.
* Of all 49 columns, 27 columns have at least 1 missing data point.
* Over half the dataset is an **object** datatype

> With the information provided on each column by the [Chicago Data Portal](https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if) I was able to preselect features I felt were irrelevant to this project. The next step would be to filter out those columns before moving forward.

In [4]:
# Specify columns to drop in new dataframe
columns_to_drop = crashes[['RD_NO', 'CRASH_DATE_EST_I', 'LANE_CNT', 'REPORT_TYPE', 'DATE_POLICE_NOTIFIED', 'STREET_NO',
                     'STREET_NO', 'STREET_NAME', 'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 
                     'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
                     'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
                     'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 
                     'LOCATION', 'BEAT_OF_OCCURRENCE', 'DAMAGE']]
columns_to_drop.head()

,RD_NO,CRASH_DATE_EST_I,LANE_CNT,REPORT_TYPE,DATE_POLICE_NOTIFIED,STREET_NO,STREET_NO,STREET_NAME,PHOTOS_TAKEN_I,STATEMENTS_TAKEN_I,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LOCATION,BEAT_OF_OCCURRENCE,DAMAGE
0,JC343143,NaN,NaN,ON SCENE,07/10/2019 06:16:00 PM,2158,2158,MARMORA AVE,NaN,NaN,...,0.0,0.0,3.0,0.0,17,4,7,POINT (-87.773287883007 41.919663832993),2515.0,"OVER $1,500"
1,JA329216,NaN,4.0,ON SCENE,06/30/2017 04:01:00 PM,8301,8301,CICERO AVE,NaN,NaN,...,0.0,0.0,3.0,0.0,16,6,6,POINT (-87.740953581987 41.741803598989),834.0,"OVER $1,500"
2,JD292400,NaN,NaN,ON SCENE,07/10/2020 10:25:00 AM,1632,1632,67TH ST,NaN,NaN,...,0.0,0.0,3.0,0.0,10,6,7,POINT (-87.585022352022 41.773455972008),331.0,"OVER $1,500"
3,JD293602,NaN,NaN,NOT ON SCENE (DESK REPORT),07/11/2020 08:30:00 AM,110,110,51ST ST,NaN,NaN,...,0.0,0.0,3.0,0.0,1,7,7,POINT (-87.622114914961 41.802118543011),224.0,$500 OR LESS
4,JD290451,NaN,NaN,ON SCENE,07/08/2020 02:15:00 PM,412,412,OHARE ST,NaN,NaN,...,0.0,0.0,1.0,0.0,14,4,7,NaN,1654.0,"OVER $1,500"


In [5]:
# Drop irrelevant columns from dataset
crashes.drop(columns=columns_to_drop, axis=1, inplace=True)
crashes.head().T

,0,1,2,3,4
CRASH_RECORD_ID,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...
CRASH_DATE,07/10/2019 05:56:00 PM,06/30/2017 04:00:00 PM,07/10/2020 10:25:00 AM,07/11/2020 01:00:00 AM,07/08/2020 02:00:00 PM
POSTED_SPEED_LIMIT,35,35,30,30,20
TRAFFIC_CONTROL_DEVICE,NO CONTROLS,STOP SIGN/FLASHER,TRAFFIC SIGNAL,NO CONTROLS,NO CONTROLS
DEVICE_CONDITION,NO CONTROLS,FUNCTIONING PROPERLY,FUNCTIONING PROPERLY,NO CONTROLS,NO CONTROLS
WEATHER_CONDITION,CLEAR,CLEAR,CLEAR,CLEAR,CLEAR
LIGHTING_CONDITION,DAYLIGHT,DAYLIGHT,DAYLIGHT,DARKNESS,DAYLIGHT
FIRST_CRASH_TYPE,TURNING,TURNING,REAR END,PARKED MOTOR VEHICLE,PARKED MOTOR VEHICLE
TRAFFICWAY_TYPE,ONE-WAY,NOT DIVIDED,FOUR WAY,DIVIDED - W/MEDIAN (NOT RAISED),DRIVEWAY
ALIGNMENT,STRAIGHT AND LEVEL,STRAIGHT AND LEVEL,STRAIGHT AND LEVEL,STRAIGHT AND LEVEL,STRAIGHT AND LEVEL


In [6]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490128 entries, 0 to 490127
Data columns (total 23 columns):
CRASH_RECORD_ID            490128 non-null object
CRASH_DATE                 490128 non-null object
POSTED_SPEED_LIMIT         490128 non-null int64
TRAFFIC_CONTROL_DEVICE     490128 non-null object
DEVICE_CONDITION           490128 non-null object
WEATHER_CONDITION          490128 non-null object
LIGHTING_CONDITION         490128 non-null object
FIRST_CRASH_TYPE           490128 non-null object
TRAFFICWAY_TYPE            490128 non-null object
ALIGNMENT                  490128 non-null object
ROADWAY_SURFACE_COND       490128 non-null object
ROAD_DEFECT                490128 non-null object
CRASH_TYPE                 490128 non-null object
INTERSECTION_RELATED_I     110608 non-null object
NOT_RIGHT_OF_WAY_I         23098 non-null object
HIT_AND_RUN_I              144603 non-null object
PRIM_CONTRIBUTORY_CAUSE    490128 non-null object
SEC_CONTRIBUTORY_CAUSE     490128 non-nul

> Once all the unwanted columns have been removed we are left with only _23 columns_ in the **crashes** dataset

In [7]:
# Transform data back to csv file
crashes.to_csv('Column_Data/crashes.csv')

### Traffic_People - Load and Explore Dataset

In [8]:
# Import People dataset into pandas dataframe
people = pd.read_csv('CSV_Datasets/Traffic_People.csv', dtype={12: str, 13: str, 20: str, 21: str, 22: str,
                                                               23: str, 24: str, 25: str, 26: str, 27: str, 29: str})
people.head().T

,0,1,2,3,4
PERSON_ID,O749947,O871921,O10018,O10038,O10039
PERSON_TYPE,DRIVER,DRIVER,DRIVER,DRIVER,DRIVER
CRASH_RECORD_ID,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...
RD_NO,JC451435,JD208731,HY484534,HY484750,HY484778
VEHICLE_ID,834816,827212,9579,9598,9600
CRASH_DATE,09/28/2019 03:30:00 AM,04/13/2020 10:50:00 PM,11/01/2015 05:00:00 AM,11/01/2015 08:00:00 AM,11/01/2015 10:15:00 AM
SEAT_NO,NaN,NaN,NaN,NaN,NaN
CITY,CHICAGO,CHICAGO,NaN,NaN,NaN
STATE,IL,IL,NaN,NaN,NaN
ZIPCODE,60651,60620,NaN,NaN,NaN


In [9]:
# View size of dataset
print(f'This dataset has {len(people)} rows and {len(people.columns)} columns\n')

# .info() to get more information on each column, i.e, dtypes, number of non-null values
people.info()

This dataset has 1082930 rows and 30 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1082930 entries, 0 to 1082929
Data columns (total 30 columns):
PERSON_ID                1082930 non-null object
PERSON_TYPE              1082930 non-null object
CRASH_RECORD_ID          1082930 non-null object
RD_NO                    1074735 non-null object
VEHICLE_ID               1061532 non-null float64
CRASH_DATE               1082930 non-null object
SEAT_NO                  221187 non-null float64
CITY                     800166 non-null object
STATE                    809138 non-null object
ZIPCODE                  730702 non-null object
SEX                      1066874 non-null object
AGE                      774106 non-null float64
DRIVERS_LICENSE_STATE    641945 non-null object
DRIVERS_LICENSE_CLASS    556876 non-null object
SAFETY_EQUIPMENT         1079716 non-null object
AIRBAG_DEPLOYED          1062470 non-null object
EJECTION                 1069572 non-null object
INJURY_CLASS

A couple of things to note:
* This dataset has **1,065,552 rows**, over double the size of the **crashes** dataset, and **30 columns**.
* Only 4 columns appear to not have any missing data point.
* Majority of the data is an **object** datatype

> Just like the **crashes** dataset we will only be using columns I feel are relevant to the model.

In [10]:
# Reassign choosen columns to people Dataframe
people = people[['CRASH_RECORD_ID', 'SEX', 'AGE', 'SAFETY_EQUIPMENT', 'DRIVER_ACTION', 'DRIVER_VISION',
                 'PHYSICAL_CONDITION', 'BAC_RESULT']]
people.head()

,CRASH_RECORD_ID,SEX,AGE,SAFETY_EQUIPMENT,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT
0,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,M,25.0,NONE PRESENT,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED
1,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,M,37.0,SAFETY BELT USED,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED
2,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,X,NaN,USAGE UNKNOWN,IMPROPER BACKING,UNKNOWN,UNKNOWN,TEST NOT OFFERED
3,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,X,NaN,USAGE UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED
4,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,X,NaN,USAGE UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED


> Of all **30 columns** only **8 columns** have been selected as _predictors_ for the model.

In [11]:
# Transform data back to csv file
people.to_csv('Column_Data/people.csv')

### Traffic_Vehicles - Load and Explore Dataset
Lastly we will be taking a look at the **Traffic_Vehicles** dataset

In [20]:
# Import Vehicles dataset into pandas dataframe
vehicles = pd.read_csv('CSV_Datasets/Traffic_Vehicles.csv', dtype={19: str, 21: str, 40: str, 41: str, 42: str,
                                                                   44: str, 48: str, 49: str, 50: str, 53: str, 
                                                                   55: str, 58: str, 59: str, 61: str, 71: str})
vehicles.head(10).T

,0,1,2,3,4,5,6,7,8,9
CRASH_UNIT_ID,829999,749947,749949,749950,871921,473653,834812,834813,837380,837381
CRASH_RECORD_ID,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,80f5b6eff349a72094c9c7a11745549caa8bd39b57cedd...,ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...,ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...
RD_NO,JD124535,JC451435,JC451435,JC451435,JD208731,JB428391,JD138756,JD138756,JD143282,JD143282
CRASH_DATE,01/22/2020 06:25:00 AM,09/28/2019 03:30:00 AM,09/28/2019 03:30:00 AM,09/28/2019 03:30:00 AM,04/13/2020 10:50:00 PM,09/09/2018 08:50:00 AM,02/03/2020 03:45:00 PM,02/03/2020 03:45:00 PM,02/06/2020 12:00:00 AM,02/06/2020 12:00:00 AM
UNIT_NO,1,1,2,3,2,2,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...
CARGO_BODY_TYPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOAD_TYPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HAZMAT_OUT_OF_SERVICE_I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MCS_OUT_OF_SERVICE_I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# View size of dataset
print(f'This dataset has {len(vehicles)} rows and {len(vehicles.columns)} columns\n')

# .info() to get more information on each column, i.e, dtypes, number of non-null values
vehicles.info()

This dataset has 1000837 rows and 72 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000837 entries, 0 to 1000836
Data columns (total 72 columns):
CRASH_UNIT_ID               1000837 non-null int64
CRASH_RECORD_ID             1000837 non-null object
RD_NO                       993096 non-null object
CRASH_DATE                  1000837 non-null object
UNIT_NO                     1000837 non-null int64
UNIT_TYPE                   999339 non-null object
NUM_PASSENGERS              149754 non-null float64
VEHICLE_ID                  977810 non-null float64
CMRC_VEH_I                  18670 non-null object
MAKE                        977805 non-null object
MODEL                       977663 non-null object
LIC_PLATE_STATE             894385 non-null object
VEHICLE_YEAR                819278 non-null float64
VEHICLE_DEFECT              977810 non-null object
VEHICLE_TYPE                977810 non-null object
VEHICLE_USE                 977810 non-null object
TRAVEL_DIRECTION     

A couple of things to note:
* This dataset has **984,626 rows**, almost as much as the **peoples** dataset, and **72 columns** which is a lot more than the first two datasets.
* This dataset has a lot more missing data per column overall 
* Majority of the data is an **object** datatype
> Just like the first two datasets we will be choosing only columns relevant to the analysis.

In [14]:
# Reassign choosen columns to vehicles Dataframe
vehicles = vehicles[['CRASH_RECORD_ID', 'VEHICLE_DEFECT', 'TRAVEL_DIRECTION', 'EXCEED_SPEED_LIMIT_I', 'MANEUVER',
                 'FIRST_CONTACT_POINT']]
vehicles.head()

,CRASH_RECORD_ID,VEHICLE_DEFECT,TRAVEL_DIRECTION,EXCEED_SPEED_LIMIT_I,MANEUVER,FIRST_CONTACT_POINT
0,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,NONE,N,NaN,STRAIGHT AHEAD,FRONT
1,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,UNKNOWN,N,NaN,STRAIGHT AHEAD,FRONT
2,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,NONE,N,NaN,PARKED,ROOF
3,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,UNKNOWN,N,NaN,PARKED,ROOF
4,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,NONE,W,NaN,STRAIGHT AHEAD,FRONT-RIGHT


> Only 6 of the columns in this dataset will be used for modeling.

In [15]:
# Transform data back to csv file
vehicles.to_csv('Column_Data/vehicles.csv')

## Join Dataframes
Now that the columns we wish to use for our analysis have been selected, the next step would be to join the new dataframes into one.

In [16]:
# Let's start by importing the new dataframes with our selected columns
# Add index_col to avoid extra index column when data loads
crashe_df = pd.read_csv('Column_Data/crashes.csv', index_col=0)
crashe_df.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,...,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_DIRECTION,WORK_ZONE_I,NUM_UNITS,LATITUDE,LONGITUDE
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,IMPROPER BACKING,UNABLE TO DETERMINE,N,NaN,2,41.919664,-87.773288
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,Y,NaN,NaN,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,S,NaN,2,41.741804,-87.740954
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,FOUR WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,FAILING TO YIELD RIGHT-OF-WAY,FAILING TO YIELD RIGHT-OF-WAY,E,NaN,3,41.773456,-87.585022
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,07/11/2020 01:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,Y,UNABLE TO DETERMINE,UNABLE TO DETERMINE,E,NaN,2,41.802119,-87.622115
4,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...,07/08/2020 02:00:00 PM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DRIVEWAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,UNABLE TO DETERMINE,UNABLE TO DETERMINE,W,NaN,2,NaN,NaN


In [17]:
vehicle_df = pd.read_csv('Column_Data/vehicles.csv', index_col=0)
vehicle_df.head()

/usr/local/anaconda3/envs/learn-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,CRASH_RECORD_ID,VEHICLE_DEFECT,TRAVEL_DIRECTION,EXCEED_SPEED_LIMIT_I,MANEUVER,FIRST_CONTACT_POINT
0,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,NONE,N,NaN,STRAIGHT AHEAD,FRONT
1,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,UNKNOWN,N,NaN,STRAIGHT AHEAD,FRONT
2,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,NONE,N,NaN,PARKED,ROOF
3,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,UNKNOWN,N,NaN,PARKED,ROOF
4,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,NONE,W,NaN,STRAIGHT AHEAD,FRONT-RIGHT


In [18]:
people_df = pd.read_csv('Column_Data/people.csv', index_col=0)
people_df.head()

/usr/local/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CRASH_RECORD_ID,SEX,AGE,SAFETY_EQUIPMENT,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT
0,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,M,25.0,NONE PRESENT,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED
1,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,M,37.0,SAFETY BELT USED,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED
2,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,X,NaN,USAGE UNKNOWN,IMPROPER BACKING,UNKNOWN,UNKNOWN,TEST NOT OFFERED
3,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,X,NaN,USAGE UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED
4,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,X,NaN,USAGE UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED


> Now we can work on joining the three dataframes. For this project I will be using the **Inner Join** method to combine the dataframes with the help of Pandas bulit in ***Merge*** function.


In [21]:
# Start by inner joining the 2 larger dataframes 'people_df' and 'vehicle_df'
# The merge function is an inner join by default so we don't need to specify 
p_v_merged = people_df.merge(crashe_df, how='inner', on=['CRASH_RECORD_ID'])
print(p_v_merged.shape)
p_v_merged.head(10).T

(1082930, 30)


,0,1,2,3,4,5,6,7,8,9
CRASH_RECORD_ID,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,dd1bce4bd6d0be4c247714dcabab44e6563c62b913229b...,4bd2ee6bb306902b99a9c2ae55cf4fcffec00879e39759...,9c03b6fcc6d72cf3ee2cb9ea754ea7e4617ab965142552...,2e7e0c1682100a200a46c25a532eda4febb00a2ae49ea9...
SEX,M,M,X,X,X,X,X,X,X,X
AGE,25,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAFETY_EQUIPMENT,NONE PRESENT,SAFETY BELT USED,USAGE UNKNOWN,USAGE UNKNOWN,USAGE UNKNOWN,USAGE UNKNOWN,USAGE UNKNOWN,USAGE UNKNOWN,USAGE UNKNOWN,USAGE UNKNOWN
DRIVER_ACTION,UNKNOWN,NONE,IMPROPER PASSING,IMPROPER BACKING,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
DRIVER_VISION,UNKNOWN,NOT OBSCURED,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
PHYSICAL_CONDITION,UNKNOWN,NORMAL,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
BAC_RESULT,TEST NOT OFFERED,TEST NOT OFFERED,TEST NOT OFFERED,TEST NOT OFFERED,TEST NOT OFFERED,TEST NOT OFFERED,TEST NOT OFFERED,TEST NOT OFFERED,TEST NOT OFFERED,TEST NOT OFFERED
CRASH_DATE,09/28/2019 03:30:00 AM,04/13/2020 10:50:00 PM,04/13/2020 10:50:00 PM,11/01/2015 05:00:00 AM,11/01/2015 08:00:00 AM,11/01/2015 10:15:00 AM,11/01/2015 11:00:00 AM,11/01/2015 12:30:00 PM,10/31/2015 04:00:00 PM,09/02/2015 02:35:00 PM
POSTED_SPEED_LIMIT,30,30,30,30,35,10,25,30,15,30


> As we can see the merge was succeful